In [1]:
import torch
from schnetpack.datasets import QM9
import schnetpack as spk
import os
from my_config import config_args
from MyModel.HGCN import RegModel
import optimizers
import numpy as np
import logging
import time
from tqdm import tqdm

# 获得训练数据

In [2]:
qm9data = QM9('./data/qm9.db', download=True,load_only=[QM9.U0])
qm9split = './data/qm9split'
print(len(qm9data))

133885


In [3]:
train, val, test = spk.train_test_split(
        data=qm9data,
        num_train=20000,
        num_val=10000,
        split_file=os.path.join(qm9split, "split20000.npz"),
    )
print(len(train),len(test),len(val))

20000 103885 10000


In [4]:
train_loader = spk.AtomsLoader(train, batch_size=256, shuffle=False)
val_loader = spk.AtomsLoader(val, batch_size=10)

In [5]:
import sys
import json
class obj(object):
    def __init__(self, dict_):
        self.__dict__.update(dict_)
args = json.loads(json.dumps(config_args), object_hook=obj)

model = RegModel(args)
optimizer = getattr(optimizers, args.optimizer)(params=model.parameters(), lr=args.lr,
                                                    weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=args.lr_reduce_freq,
    gamma=float(args.gamma)
)
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
logging.info(f"Total number of parameters: {tot_params}")
device = torch.device('cuda')
# Train model
t_total = time.time()
model = model.to(device)
loss_fun = torch.nn.MSELoss()
# torch.set_printoptions(threshold=np.inf)
for epoch in range(args.epochs):
    loss_sum,n,t = 0,0,0.0
    counter = 0
    for input in tqdm(train_loader):
        # counter+=1
        # if counter>10:
        #     sys.exit(0)
        for key in input:
            input[key] = input[key].to(torch.device('cuda'))
        t = time.time()
        model.train()
        optimizer.zero_grad()
        y_hat = model(input)
        # print('y_hat',y_hat[:2])
        # print('input[QM9.U0]',input[QM9.U0][:2])
        loss = loss_fun(y_hat,input[QM9.U0])
        # curvatures = list(model.get_submodule('curvatures'))
        # print(curvatures)

        # print(loss)
        loss.backward()
        loss_sum += loss
        n += 1
        if args.grad_clip is not None:
            max_norm = float(args.grad_clip)
            all_params = list(model.parameters())
            for param in all_params:
                torch.nn.utils.clip_grad_norm_(param, max_norm)
        optimizer.step()
    lr_scheduler.step()
    if (epoch + 1) % args.log_freq == 0:
        str = " ".join(['Epoch: {:04d}'.format(epoch + 1),
                               'lr: {}'.format(lr_scheduler.get_last_lr()[0]),
                               'loss: {:.4f}'.format(loss_sum/n),
                               'time: {:.4f}s'.format(time.time() - t)
                               ])
        print(str)
        logging.info(str)
        curvatures = list(model.get_submodule('curvatures'))
        # print(curvatures)



    # if (epoch + 1) % args.eval_freq == 0:
    #     model.eval()
    #     embeddings = model.encode(data['features'], data['adj_train_norm'])
    #     val_metrics = model.compute_metrics(embeddings, data, 'val')
    #     if (epoch + 1) % args.log_freq == 0:
    #         logging.info(" ".join(['Epoch: {:04d}'.format(epoch + 1), format_metrics(val_metrics, 'val')]))
    #     if model.has_improved(best_val_metrics, val_metrics):
    #         best_test_metrics = model.compute_metrics(embeddings, data, 'test')
    #         best_emb = embeddings.cpu()
    #         if args.save:
    #             np.save(os.path.join(save_dir, 'embeddings.npy'), best_emb.detach().numpy())
    #         best_val_metrics = val_metrics
    #         counter = 0
    #     else:
    #         counter += 1
    #         if counter == args.patience and epoch > args.min_epochs:
    #             logging.info("Early stopping")
    #             break

"""
euclid
Epoch: 0020 lr: 0.005 loss: 377.5419 time: 0.0045s
Epoch: 0040 lr: 0.0025 loss: 66.3344 time: 0.0000s
Epoch: 0060 lr: 0.00125 loss: 46.7183 time: 0.0199s

hyperbolid
Epoch: 0012 lr: 0.01 loss: 33.4861 time: 0.0595s
Epoch: 0020 lr: 0.005 loss: 6.2979 time: 0.0600s
Epoch: 0040 lr: 0.0025 loss: 4.1167 time: 0.0697s
"""

100%|██████████| 79/79 [00:32<00:00,  2.46it/s]


Epoch: 0001 lr: 0.01 loss: 23098756.0000 time: 0.0565s


100%|██████████| 79/79 [00:29<00:00,  2.68it/s]


Epoch: 0002 lr: 0.01 loss: 5280187.5000 time: 0.0610s


100%|██████████| 79/79 [00:32<00:00,  2.46it/s]


...

Epoch: 0093 lr: 0.000625 loss: 5.7840 time: 0.0572s


100%|██████████| 79/79 [00:32<00:00,  2.42it/s]


Epoch: 0094 lr: 0.000625 loss: 6.2939 time: 0.0583s


 22%|██▏       | 17/79 [00:07<00:26,  2.37it/s]


KeyboardInterrupt: 

In [ ]:
# from manifolds.hyperboloid import Hyperboloid
# import torch
# size = (2,5,10)
# a = 2*torch.ones(size)
# a = torch.concat([torch.zeros((2,5,1)),a],dim=2)
# print(a)
# hyper = Hyperboloid()
# c = 1
# a_H = hyper.expmap0(a,c)  #
# print(a_H)
# a_H_p = hyper.proj(a_H,c)  #对双曲空间的向量没有影响
# print(a_H_p)

# a_tan = hyper.logmap0(a_H,c)
# print(a_tan)
#
# a_tan_p = hyper.proj_tan0(a_tan,c)  #  对已经在o点切空间的向量没有影响
# print(a_tan_p)

# a_tan_a = hyper.logmap(a_H,a_H,c)  # 向量在其切空间是0
# print(a_tan_a)



